이번 챕터에서는 문자 단위가 아니라 RNN의 입력 단위를 단어 단위로 사용합니다. 그리고 단어 단위를 사용함에 따라서 Pytorch에서 제공하는 임베딩 층(embedding layer)를 사용하겠습니다.

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

In [2]:
sentence = "Repeat is the best medicine for memory".split()

In [3]:
vocab = list(set(sentence))
print(vocab)

['Repeat', 'memory', 'for', 'medicine', 'is', 'the', 'best']


In [7]:
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0
print(word2index)

{'Repeat': 1, 'memory': 2, 'for': 3, 'medicine': 4, 'is': 5, 'the': 6, 'best': 7, '<unk>': 0}


In [8]:
print(word2index['memory'])

2


In [9]:
# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'Repeat', 2: 'memory', 3: 'for', 4: 'medicine', 5: 'is', 6: 'the', 7: 'best', 0: '<unk>'}


In [10]:
print(index2word[2])

memory


In [17]:
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    print(encoded)
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    print(input_seq)
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    print(label_seq)

    return input_seq, label_seq


In [18]:
X, Y = build_data(sentence, word2index)
print(X)
print(Y)

[1, 5, 6, 7, 4, 3, 2]
tensor([[1, 5, 6, 7, 4, 3]])
tensor([[5, 6, 7, 4, 3, 2]])
tensor([[1, 5, 6, 7, 4, 3]])
tensor([[5, 6, 7, 4, 3, 2]])


# 모델 구현

In [19]:
class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함. 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [20]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [21]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [22]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[-0.3405,  0.0850, -0.1027,  0.1218,  0.0128, -0.1233,  0.0732,  0.1191],
        [-0.2586,  0.0179, -0.2560, -0.0113,  0.1338,  0.1448,  0.1257,  0.2675],
        [-0.4598,  0.0859, -0.0519, -0.0527,  0.1043, -0.0602, -0.1958,  0.0568],
        [-0.5010,  0.4520, -0.3220, -0.2492, -0.0501,  0.0827, -0.2328,  0.0918],
        [-0.3727,  0.0009, -0.0794,  0.0712,  0.0288, -0.2563,  0.0164, -0.1110],
        [-0.3955,  0.0592, -0.1777,  0.3861, -0.0700,  0.1095, -0.0174,  0.4822]],
       grad_fn=<ViewBackward>)


In [23]:
# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [24]:
# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0831 
Repeat for best medicine Repeat for best

[41/201] 1.4715 
Repeat for the best medicine for memory

[81/201] 0.8339 
Repeat is the best medicine for memory

[121/201] 0.4438 
Repeat is the best medicine for memory

[161/201] 0.2507 
Repeat is the best medicine for memory

[201/201] 0.1515 
Repeat is the best medicine for memory

